In [ ]:
# O objetivo aqui vai ser arrumar os dados para backtest

In [1]:
import pandas as pd
import numpy as np
# https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html

In [ ]:
# Algumas questões sobre markdown
# https://tex.stackexchange.com/questions/130510/write-text-correctly-in-equations #\mathrm and its friends
# https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html
# https://www.ibm.com/support/knowledgecenter/en/SSGNPV_1.1.3/dsx/markd-jupyter.html

The magic formula ranks companies based on two factors: return on capital and earnings yield.

#### Return on Capital
$\textrm{ROC} = \textrm{EBIT}/\textrm{(Net Working Capital + Net Fixed Assets)}$

$\begin{split}
\textrm{Net Fixed Assets} & = \textrm{Total Assets} \\
 & - \textrm{Total Current Assets} \\
 & - \textrm{Total Intangibles & Goodwill}
\end{split}$

#### Earnings yield
$\textrm{Earnings Yield} = \textrm{EBIT} / \textrm{Enterprise Value}$

$\begin{split}
\textrm{Enterprise Value} & = \textrm{Market Value of Equity} \\
 & + \textrm{Net Interest-Bearing Debt}
\end{split}$

In [126]:
def org_columns(df):
    '''
    Organiza os dataframes com fundamentos:
        Renomeia columns
        Seleciona apenas tickers de interesse
    '''
    tickers = df.columns.str.extract('(.*)$')[0]
    df.columns = tickers
    on = tickers[tickers.str.contains('.\D3$')]
    pn = tickers[tickers.str.contains('.\D4$')]
    sto_tick = pd.concat([on, pn], ignore_index=True)
    df = df[sto_tick]
    df = df[df.index.str.contains("Mar|Jun|Sep|Dec")]
    df.columns.name = "ticker"
    return df

In [127]:
# EBIT
ebit_ati = pd.read_excel('../../data/economatica_1/ebit_mensal__ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ebit_can = pd.read_excel('../../data/economatica_1/ebit_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ebit_can.drop(["Jan-86", "Feb-86"], inplace=True)

ebit = pd.concat([ebit_ati,ebit_can],axis=1,sort=False)
ebit = org_columns(ebit)

In [128]:
# Net Working Capital
nwc_ati = pd.read_excel('../../data/economatica_1/working_capital_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
nwc_can = pd.read_excel('../../data/economatica_1/working_capital_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
nwc_can.drop(["Jan-86", "Feb-86"], inplace=True)

nwc = pd.concat([ebit_ati,ebit_can],axis=1,sort=False)
nwc = org_columns(nwc)

In [129]:
# Total Assets
ta_ati = pd.read_excel('../../data/economatica_1/total_assets_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ta_can = pd.read_excel('../../data/economatica_1/total_assets_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ta_can.drop(["Jan-86", "Feb-86"], inplace=True)

ta = pd.concat([ta_ati,ta_can],axis=1,sort=False)
ta = org_columns(ta)

In [130]:
# Ativo Circulante
ac_ati = pd.read_excel('../../data/economatica_1/ativo_circulante_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ac_can = pd.read_excel('../../data/economatica_1/ativo_circulante_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ac_can.drop(["Jan-86", "Feb-86"], inplace=True)

ac = pd.concat([ac_ati,ac_can],axis=1,sort=False)
ac = org_columns(ac)

In [131]:
# Passivo Circulante
pc_ati = pd.read_excel('../../data/economatica_1/passivo_circulante_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
pc_can = pd.read_excel('../../data/economatica_1/passivo_circulante_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
pc_can.drop(["Jan-86", "Feb-86"], inplace=True)

pc = pd.concat([pc_ati,pc_can],axis=1,sort=False)
pc = org_columns(pc)

In [132]:
# Total Current Assets
# tca não perdeu nada nessa operação quanto ao não uso de fill_value em relação a ac-pc
# lembrar que nessas operações o número de colunas vai ser o maior possível, tipo com concatenar
# tca = ac - pc
tca = ac.sub(pc,fill_value=0)

In [133]:
#
tot_na = tca.isnull().sum().sum()
# o padrão de qualquer função sobre matriz vai sempre sempre ser linha antes, coluna depois
l, c = tca.shape

tot = l*c
print(tot)
print(tot_na)
print(tot_na/tot)

134865
84883
0.6293923553182812


In [134]:
# Goodwill
gw_ati = pd.read_excel('../../data/economatica_1/goodwill_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
gw_can = pd.read_excel('../../data/economatica_1/goodwill_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
# pc_can.drop(["Jan-86", "Feb-86"], inplace=True)

gw = pd.concat([gw_ati,gw_can],axis=1,sort=False)
gw = org_columns(gw)

In [135]:
x = gw.loc["Mar-19"]
y = ~x.isnull()
y.sum()

2

In [138]:
x = ~gw.isna()
x.sum().sum()

28879

In [139]:
# Net Fixed Assets
# nfa = ta - tca - gw
# lembrar que a ordem de precedência é a sub tca e só depois a sub gw
# nfa2 = ta.sub(tca, fill_value=0).sub(gw, fill_value=0)
# nfa.equals(nfa2) # vai dar False pois devem ser diferentes

nfa = ta.sub(tca, fill_value=0).sub(gw, fill_value=0)

In [140]:
# Market Value of Equity
mve_ati = pd.read_excel('../../data/economatica_1/valor_mercado_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
mve_can = pd.read_excel('../../data/economatica_1/valor_mercado_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
mve_can.drop(["Jan-86", "Feb-86"], inplace=True)

mve = pd.concat([mve_ati,mve_can],axis=1,sort=False)
mve = org_columns(mve)

In [141]:
# Net Interest-Bearing Debt
nibd_ati = pd.read_excel('../../data/economatica_1/net_debt_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
nibd_can = pd.read_excel('../../data/economatica_1/net_debt_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
nibd_can.drop(["Jan-86", "Feb-86"], inplace=True)

nibd = pd.concat([nibd_ati,nibd_can],axis=1,sort=False)
nibd = org_columns(nibd)

In [142]:
# Enterprise Level
# ev1 dá bem diferente de ev2
# ev1 = mve + nibd
# ev2 = mve.add(nibd, fill_value=0)

ev = mve.add(nibd, fill_value=0)

In [143]:
# Return on Capital (ROC)
# roc = ebit / (nwc + nfa)
roc = ebit.div(nwc.add(nfa,fill_value=0),fill_value=0)

In [144]:
# Earnings Yield
ey = ebit.div(ev, fill_value=0)

In [148]:
ey

ticker,AALR3,ABCB3,ABCB4,ABEV3,ABYA3,ACES3,ACES4,ACGU3,ADHM3,AEDU3,...,WISA3,WISA4,WIZS3,WLMM3,WLMM4,WMBY3,WMBY4,WWOW3,YDUQ3,ZIVI4
Date,,,,,,,,,,,,,,,,,,,,,
Mar-86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Jun-86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sep-86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dec-86,NaN,NaN,NaN,0.346678,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.253990,0.253990,0.000000e+00,0.000000e+00,NaN,NaN,NaN
Mar-87,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000000,0.000000,0.000000e+00,0.000000e+00,NaN,NaN,NaN
Jun-87,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000000,0.000000,0.000000e+00,0.000000e+00,NaN,NaN,0.000000
Sep-87,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000000,0.000000,0.000000e+00,0.000000e+00,NaN,NaN,0.000000
Dec-87,NaN,NaN,NaN,-0.224333,NaN,0.000000,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,1.279382,1.279382,-3.377197e-03,-3.377197e-03,NaN,NaN,0.000000
Mar-88,NaN,NaN,NaN,0.000000,NaN,0.000000,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000000,0.000000,0.000000e+00,0.000000e+00,NaN,NaN,0.000000


In [150]:
# ey.loc["Mar-18"]
ey_ = ey.loc["Jun-19"]
roc_ = roc.loc["Mar-19"]

In [152]:
ey_.sort_values(ascending=False)

ticker
FIGE3    179.666667
FIGE4    179.666667
CALI3     54.563107
CALI4     54.563107
CMSA4      1.098619
CMSA3      1.098619
MNPR3      0.624504
STTR3      0.584374
CEGR3      0.462138
CORR3      0.453794
CORR4      0.453794
CEAB3      0.435736
STKF3      0.311146
VIVA3      0.291689
RSUL4      0.265535
RSUL3      0.265535
GPAR3      0.238085
INNT3      0.228256
ODER3      0.194433
ODER4      0.194433
MNPR4      0.184226
PTNT3      0.172129
PTNT4      0.172129
AGRO3      0.171524
CNSY3      0.165935
CEBR3      0.156307
QUSW3      0.145402
SMFT3      0.144353
NUTR3      0.144056
CASN4      0.140790
            ...    
UOLL3           NaN
UOLL4           NaN
VAGV3           NaN
VAGV4           NaN
VASP3           NaN
VCPA4           NaN
VEMG4           NaN
VGOR3           NaN
VGOR4           NaN
VIGR3           NaN
VINE3           NaN
VIVO3           NaN
VIVO4           NaN
VPSC3           NaN
VPSC4           NaN
VPTA3           NaN
VPTA4           NaN
VRLM4           NaN
VSMA3        

In [47]:
# criar df com dictionary e um index e usar expressões lógicas
df1 = pd.DataFrame({'A': [12, 44, 17, 1],
                    'B': [4, 8, 100, -3],
                    'C': [47, 68, 52, 1],
                    'D': [0, 0, 0, 0]},
                    index=[0, 1, 2, 3])
df2 = pd.DataFrame({'A': [1, 2, 3, 4],
                    'B': [-10, 4, 5, 13],
                    'C': [12, 23, 10, 100],
                    'D': [1, 1, 1, 1]},
                    index=[0, 1, 2, 3])
df3 = pd.DataFrame({'A': [1, 2, 3, 4],
                    'B': [-10, np.nan, 5, 13],
                    'C': [np.nan, 23, 10, 100],
                    'D': [1, 1, 1, 1]},
                    index=[0, 1, 2, 3])
df4 = pd.DataFrame({'E': [10, 10, 10, 10],
                    'F': [10, 10, 10, 10]},
                    index=[0, 1, 2, 3])

In [7]:
df1+df3

,A,B,C,D
0,13,-6.0,NaN,1
1,46,NaN,91.0,1
2,20,105.0,62.0,1
3,5,10.0,101.0,1


In [39]:
print(
    df1.add(df3,fill_value=0)
)
print(
    df1+df3
)
# perceber como o fill value serve tanto para NAs no primeiro como no segundo dataframes
print(
    df3.add(df1,fill_value=0)
)

    A      B      C  D
0  13   -6.0   47.0  1
1  46    8.0   91.0  1
2  20  105.0   62.0  1
3   5   10.0  101.0  1
    A      B      C  D
0  13   -6.0    NaN  1
1  46    NaN   91.0  1
2  20  105.0   62.0  1
3   5   10.0  101.0  1
    A      B      C  D
0  13   -6.0   47.0  1
1  46    8.0   91.0  1
2  20  105.0   62.0  1
3   5   10.0  101.0  1


In [49]:
# concatena as colunas mas como não são coincidentes fica tudo NA
df1+df4
# concatena as colunas e cada coluna fica igual pois são não coincidentes
df1.add(df4, fill_value=0)

,A,B,C,D,E,F
0,12.0,4.0,47.0,0.0,10.0,10.0
1,44.0,8.0,68.0,0.0,10.0,10.0
2,17.0,100.0,52.0,0.0,10.0,10.0
3,1.0,-3.0,1.0,0.0,10.0,10.0


In [54]:
df1+df2+df3

,A,B,C,D
0,14,-16.0,NaN,2
1,48,NaN,114.0,2
2,23,110.0,72.0,2
3,9,23.0,201.0,2


In [59]:
df1.add(df2).sub(df3)

,A,B,C,D
0,12,4.0,NaN,0
1,44,NaN,68.0,0
2,17,100.0,52.0,0
3,1,-3.0,1.0,0


In [88]:
A = pd.DataFrame({'A':[10]})
B = pd.DataFrame({'A':[1]})
C = pd.DataFrame({'A':[5]})

In [17]:
# criar df com dictionary e um index e usar expressões lógicas
# engraçado como em dataframes True+True = True, False+False=False, True+False=True
df1 = pd.DataFrame({'A': [True, False, False, True],
                    'B': [False, True, True, False],
                    'C': [False, False, True, False],
                    'D': [True, False, True, True]},
                    index=[0, 1, 2, 3])
df2 = pd.DataFrame({'A': [True, True, True, True],
                    'B': [True, True, True, True],
                    'C': [True, True, True, True],
                    'D': [False, False, False, False]},
                    index=[0, 1, 2, 3])

In [19]:
df1&df2

,A,B,C,D
0,True,False,False,False
1,False,True,False,False
2,False,True,True,False
3,True,False,False,False


In [92]:
# A ordem de precedência das operações obviamente é add primeiro e depois div
B.div(A).add(C) # (1/10)+5 = 5.1
#B.add(C).div(A) # (1+5)/10 = 0.6

,A
0,5.1


In [119]:
A.div(C)

,A
0,2.0
